In [2]:
import os
import pandas as pd
import numpy as np

In [48]:
list_df = []
path = os.getcwd() + "/csvfiles/"
t = 0 #week number
for file in os.listdir(path.replace("\\","/")):
    if file.startswith('tr'):
        new_df = pd.read_csv(path + file)[['SHOP_WEEK', 'QUANTITY', 'SPEND', 'PROD_CODE', 'STORE_CODE']]
        new_df['SHOP_WEEK'] = np.ones(len(new_df[['SHOP_WEEK']]))*t
        new_df['SHOP_WEEK'] = new_df['SHOP_WEEK'].astype(int)
        list_df.append(new_df)
        t+=1
df = pd.concat(list_df,axis=0)
df = df[df['QUANTITY'] > 0].copy()

#On ne prend que les magasins avec le plus de transactions
nb_stores = 10
top_stores = df['STORE_CODE'].value_counts().head(nb_stores).index.tolist()
df_top = df[df['STORE_CODE'].isin(top_stores)]

nb_products = 50
top_products = df_top['PROD_CODE'].value_counts().head(nb_products).index.tolist()
df_top = df_top[df_top['PROD_CODE'].isin(top_products)]

#On prend le prix unitaire par transaction
df_top['UNIT_PRICE'] = df_top['SPEND'] / df_top['QUANTITY'] 
df_final = df_top.pivot_table(
    index=['SHOP_WEEK', 'STORE_CODE'],
    columns='PROD_CODE',
    values='UNIT_PRICE',
    aggfunc='mean'
)
df_final.reset_index(inplace=True)

#fill the NANs with forward method
df_final.ffill(inplace=True)
df_final.bfill(inplace=True)

#replace 'store_code' by store number
keys = pd.unique(df_final['STORE_CODE'])
values = np.arange(len(pd.unique(df_final['STORE_CODE'])))
df_final['STORE_CODE'].replace(keys, values, inplace=True)
df_final

#keep only products with enough variance in price
v = df_final.var()
products = v[v>1e-2]
df_final = df_final[products.index]
df_final

PROD_CODE,SHOP_WEEK,STORE_CODE,PRD0900173,PRD0900531,PRD0900679,PRD0901265,PRD0901878,PRD0902540,PRD0903052
0,0,0,3.310000,3.430,2.39,1.8750,0.896667,0.145,1.0975
1,0,1,3.310000,3.430,2.39,1.8750,0.896667,0.145,1.4500
2,0,2,1.046667,1.000,2.39,1.5350,0.896667,0.145,0.7650
3,0,3,1.460000,1.000,2.39,2.0900,0.896667,0.145,0.6050
4,0,4,1.680000,1.590,2.39,1.9325,0.896667,0.145,0.7600
...,...,...,...,...,...,...,...,...,...
1145,114,5,2.060000,2.350,0.79,1.9700,0.660000,0.290,0.7075
1146,114,6,2.060000,1.025,0.79,1.5200,0.660000,0.290,0.6000
1147,114,7,2.530000,1.025,0.77,1.4275,0.660000,0.360,0.7020
1148,114,8,2.530000,1.750,0.54,1.7150,0.877500,0.500,0.4500


In [61]:

np.argmin([2,2,1,2222,2])


2

In [21]:
df_final = df_top.pivot_table(
    index=['SHOP_WEEK', 'STORE_CODE'],
    columns='PROD_CODE',
    values='UNIT_PRICE',
    aggfunc='mean'
)
df_final.reset_index(inplace=True)

#fill the NANs with forward method
df_final.ffill(inplace=True)
df_final.bfill(inplace=True)

#replace 'store_code' by store number
key = pd.unique(df_final['STORE_CODE'])
values = np.arange(len(pd.unique(df_final['STORE_CODE'])))
#df_final['STORE_CODE'] = pd.to_numeric(df_final['STORE_CODE'])
df_final['STORE_CODE'].replace(key, values, inplace=True)
df_final

PROD_CODE,SHOP_WEEK,STORE_CODE,PRD0900121,PRD0900159,PRD0900173,PRD0900202,PRD0900254,PRD0900385,PRD0900531,PRD0900679,...,PRD0903788,PRD0903867,PRD0903993,PRD0904044,PRD0904245,PRD0904250,PRD0904358,PRD0904887,PRD0904962,PRD0904976
0,0.0,0,0.98,1.83,3.310000,1.37,0.85,0.8,3.430,2.39,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
1,0.0,1,0.98,1.83,3.310000,1.37,0.85,0.8,3.430,2.39,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
2,0.0,2,0.98,1.83,1.046667,1.37,0.85,0.8,1.000,2.39,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
3,0.0,3,0.98,1.83,1.460000,1.37,0.85,0.8,1.000,2.39,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
4,0.0,4,0.98,1.83,1.680000,1.37,0.85,0.8,1.590,2.39,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,114.0,5,0.98,1.83,2.060000,1.37,0.85,0.8,2.350,0.79,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
1146,114.0,6,0.98,1.83,2.060000,1.37,0.85,0.8,1.025,0.79,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
1147,114.0,7,0.98,1.83,2.530000,1.37,0.85,0.8,1.025,0.77,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
1148,114.0,8,0.98,1.83,2.530000,1.37,0.85,0.8,1.750,0.54,...,0.75,1.93,1.8,0.56,1.16,2.49,1.54,1.02,2.09,1.49
